### 제미나이 API 준비

In [1]:
# 패키지 설치
!pip install -q -U google-generativeai

In [2]:
from google.colab import userdata
import google.generativeai as genai

# 환경 변수 준비(좌측 하단의 열쇠 아이콘으로 GOOGLE_API_KEY 설정)
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

### 이미지 질의 응답

In [3]:
import PIL.Image

# 이미지 불러오기
image = PIL.Image.open("image.jpg")
image

Output hidden; open in https://colab.research.google.com to view.

In [4]:
# 모델 준비
model = genai.GenerativeModel(
    "models/gemini-1.5-flash"
)

In [5]:
# 추론 실행
response = model.generate_content([
    image,
    "이것은 무슨 이미지입니까？"
])
print(response.text)

이것은 나무 테이블 위에 누워 있는 보더 콜리 강아지의 사진입니다.  강아지는 검은색과 흰색의 털을 가지고 있으며, 카메라를 바라보고 있습니다. 배경에는 나무 바닥과 침대가 보입니다.


### File API를 사용한 이미지 질의 응답

In [ ]:
# 이미지를 File API 서버로 업로드
image_file = genai.upload_file(path="image.jpg")

# 확인
print("name:", image_file.name)
print("display_name:", image_file.display_name)
print("uri:", image_file.uri)

name: files/xj095l2ni58a
display_name: image.jpg
uri: https://generativelanguage.googleapis.com/v1beta/files/xj095l2ni58a


In [ ]:
# 추론 실행
response = model.generate_content(
    [
        image_file,
        "이것은 무슨 이미지입니까?"
    ]
)
print(response.text)

 이것은 고양이가 TV 앞에 앉아 있는 이미지입니다. 고양이가 흰색과 검은색으로 되어 있습니다. 고양이는 매우 귀엽습니다.


In [ ]:
# 파일 삭제
genai.delete_file(image_file.name)

### 음성 질의 응답

In [ ]:
# 파일 업로드
audio_file = genai.upload_file(path="sample.wav")

In [ ]:
# 모델 준비
model = genai.GenerativeModel(
    "models/gemini-1.5-pro"
)

# 내용 유추
response = model.generate_content(
    [
        "다음 음성의 내용이 무엇에 관한 것 인지 한국어로 간단하게 유추해주세요.",
        audio_file
    ]
)
print(response.text)

과거에 누군가와 짧은 만남을 가졌지만, 그는 신들의 말석에 자리한 미천한 존재였던 것 같습니다. 화자는 그가 그런 선택을 했다는 사실에 놀라워하며, 다음 세대에서도 그는 많은 사람들을 만나게 될 것이라고 예상합니다. 

전체적으로 어떤 존재의 짧은 생과 그가 남긴 영향에 대해 이야기하는 듯합니다. 



In [ ]:
# 음성을 텍스트로 변환
response = model.generate_content(
    [
        "다음 음성을 한국어로 요약 없이 변환해주세요.",
        audio_file
    ]
)
print(response.text)

전에 그와 잔을 기울일 때까지만 해도 나는 신들의 말석에 자리한 일개 그림자 무사에 불과했어. 그가 그러한 선택을 했단 말이지? 허나 다음 세대에서도 그는 분명 계속해서 많은 사람을 만나게 되겠군. 


In [ ]:
# 파일 삭제
genai.delete_file(audio_file.name)

### 동영상 질의 응답

In [ ]:
# 동영상 다운로드
!wget https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4

--2024-10-22 13:24:25--  https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4
Resolving download.blender.org (download.blender.org)... 172.67.14.163, 104.22.65.163, 104.22.64.163, ...
Connecting to download.blender.org (download.blender.org)|172.67.14.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64657027 (62M) [video/mp4]
Saving to: ‘BigBuckBunny_320x180.mp4’

BigBuckBunny_320x18 100%[===================>]  61.66M  21.5MB/s    in 2.9s    

2024-10-22 13:24:28 (21.5 MB/s) - ‘BigBuckBunny_320x180.mp4’ saved [64657027/64657027]



In [ ]:
# 동영상 업로드
video_file = genai.upload_file(path="BigBuckBunny_320x180.mp4")
print("Completed upload:", video_file.uri)

Completed upload: https://generativelanguage.googleapis.com/v1beta/files/1p7clsruy6c0


In [ ]:
import time

# API가 파일을 수신했는지 확인
while video_file.state.name == "PROCESSING":
    print("Waiting for video to be processed.")
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)
print("Video processing complete:", video_file.uri)

Waiting for video to be processed.
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/1p7clsruy6c0


In [ ]:
# 모델 준비
model = genai.GenerativeModel(
    model_name="models/gemini-1.5-pro"
)

# 추론 실행
response = model.generate_content(
    ["이 동영상을 한국어로 설명해주세요.", video_file],
    request_options={"timeout": 600} # 타임 아웃 지정
)
print(response.text)

동영상은 빅 벅 버니라는 애니메이션 영화의 오프닝 시퀀스입니다.

장면은 푸른 하늘에 핑크색 구름이 떠 있는 아름다운 시골로 시작됩니다. 카메라가 푸른 숲, 푸른 초원, 개울로 가득한 풍경으로 이동합니다.

나무 위에 앉아 있는 파란 새가 잠에서 깨어나 "THE PEACH OPEN MOVIE PROJECT PRESENTS"라는 텍스트가 나타납니다. 

그런 다음 카메라는 나무 뿌리에 위치한 작은 동굴로 이동하여 "BIG BUCK BUNNY"라는 제목이 화면에 나타납니다.

동굴 안에서 토실토실한 토끼가 잠에서 깨어 기지개를 켜고 밖으로 나옵니다. 토끼는 초원을 산책하며 꽃향기를 맡고 나비와 놀다가 땅에 떨어진 사과를 발견합니다.

토끼는 사과를 주워 먹으려고 하지만 다람쥐, 여우, 친칠라가 다가와 비웃습니다. 다람쥐는 토끼에게 사과를 먹으려면 사과 나무에 올라가서 직접 따야 한다고 놀립니다.

토끼는 나무에 올라가 사과를 따려고 하지만 너무 뚱뚱해서 올라갈 수 없습니다. 다시 땅으로 내려오자, 동물 친구들이 계속해서 토끼를 비웃습니다.

화가 난 토끼는 나무 막대기를 집어 들고 창과 활을 만듭니다. 토끼는 목표물을 향해 창을 던지지만, 빗나가 동물 친구들을 위협합니다. 동물들은 겁에 질려 도망칩니다.

토끼는 활을 쏘아 나무에 사과를 맞춰 땅에 떨어뜨립니다. 토끼는 만족스러운 표정으로 사과를 집어 들고 동물 친구들이 숨어 있는 것을 바라봅니다. 

동영상은 토끼가 카메라를 향해 윙크하면서 끝납니다.

동영상의 크레딧에는 감독, 아트 디렉터, 리드 아티스트, 캐릭터 리거, 캐릭터 애니메이터, 기술 감독, 소프트웨어 개발자, 음악 및 사운드 디자인, 원작 스토리 및 대본, 스토리보드, 컬러 아트워크, 애니메이션 편집, 캐릭터 디자인, 캐릭터 모델링, 캐릭터 리깅, 캐릭터 영감, 매트 페인팅, 텍스처 페인팅, 환경 디자인 및 레이아웃, 나무 모델링 및 스크립팅, 환경 모델링 및 셰이딩, 환경 및 소품 애니메이션, 모피 및 깃털, 셰이딩 및 조명, 합성, 기술 감독,

In [ ]:
# 파일 삭제
genai.delete_file(video_file.name)